In [ ]:
# import os
# os.environ["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
# os.environ["DEV_MODE"] = "True"
# os.environ["TEST_EXTERNAL_REGISTRY"] = "k3d-registry.localhost:5800"

In [3]:
# new
from os import environ as env
env["ORCHESTRA_DEPLOYMENT_TYPE"] = "remote"
(environment := env.get("ORCHESTRA_DEPLOYMENT_TYPE", "python"))

'remote'

In [4]:
# syft absolute
import syft as sy

In [5]:
server = sy.orchestra.launch(
    name="bigquery-high",
    dev_mode=True,
    server_side_type="high",
    port="8080",
    n_consumers=1,  # How many workers to be spawned
    create_producer=True,  # Can produce more workers
)

In [6]:
high_client = sy.login(
    url="http://localhost:8080",
    email="data_scientist@openmined.org",
    password="verysecurepassword",
)

Logged into <syft-dev-server: High side Datasite> as <data_scientist@openmined.org>


In [7]:
job = high_client.code.popular(blocking=False)
job

syft.service.job.job_stash.Job

In [8]:
result = job.wait().get()

SyftWarning: This is a placeholder object, the real data lives on a different server and is not synced.

In [9]:
assert len(result) == 10000

In [10]:
server.land()

Shutdown not implemented for the deployment type:DeploymentType.REMOTE
